# Armita Bahrudi - 810100591

Artificial Intelligence - CA#01: *Genetics* - fall 1402 \
In this notebook, we will implement a genetic algorithm to solve curve fitting optimization problem.

## Genetic Algorithm introduction

genetic algorithm is a method for solving optimization problems that based on the process that drives biological evolution. \
in the following steps, this method is used to fit a polynomial function to the given points.


In [21]:
import random

### Parameters


In [22]:
population_size = 250
num_of_gens = 100
mutation_rate = 0.1
cross_over_rate = 0.8
perfect_result = 1.0

## Take Input

In [23]:
degree = int(input())
coef_range = (int(input()), int(input()))
target_points_number = 0
target_points = []
while True:
    x, y = input(), input()
    if x == "" or y == "":
        break
    new_point = (int(x), int(y))
    target_points.append(new_point)
    target_points_number += 1

## Population Initialization

In this section, we will generate a number of chromosomes (set of coefficients) randomly to be our initial population.

In [24]:
def initialize_pop():
    return [random.sample(range( coef_range[0], coef_range[1] + 1) , degree + 1 ) for _ in range(population_size)]

## Check Fitness

In this section, we define a ```check_fitness()``` function that shows us the degree of desirability of each chromosome. \
The fitness of the chromosome is calculated using the following formula:
$$ fitness =  1  /  (1 + (\sum_{i=1}^{target\_points\_number} (|f(x_{i}) - y_{i}|))/ target\_points\_number) $$
$f(x)$ used in the formula is a polynomial with the coefficients of each chromosome, and $x_{i}$ , $y_{i}$ are the coordinates of a point in target_points.\
closer fitness value to 1 is better for us. we average the difference of answer and points so the fitness does not get usually too close to zero and we do not lose the calculations accuracy.


In [25]:
def check_fitness(chromosome):
    error = 0
    for x, y in target_points:
        f_x = 0
        for i in range(degree + 1):
            f_x += ( x ** i ) * chromosome[i]
        error += abs(f_x - y)
    fitness = 1/(1 + (error / target_points_number))
    return fitness

## Selection

In this section, we use ```roulette method``` (one of stochastic selection method in GA) to select the chromosomes that are supposed to remain. \
The following figure contains the random number generated using Uniform(0, 1). Important to note that number of chromosomes selected for cross-over is controlled by the cross-over rate (Pc) where the minimum is 0 and the maximum is 1.
For instance, we determine Pc = 0.25. It means that the chromosomes where the random number is less than 0.25 will be the parents in the cross-over. 

In [26]:
def selection(chromosomes, total, ch_fitnesses):
    cumulative_probability = calculate_cumulative_p(total, ch_fitnesses)
    return chose_new_chromosomes(chromosomes, cumulative_probability)

def calculate_cumulative_p(total, ch_fitnesses):
    cumulative_probability = []
    p_sum = 0 
    for f in ch_fitnesses:
        p = f / total
        cumulative_probability.append(p_sum + p)
        p_sum += p
    return cumulative_probability

def chose_new_chromosomes(chromosomes, cumulative_probability):
    new_chromosomes = []
    for _ in chromosomes:
       chose_parameter = random.uniform(0,1)
       for i in range(0, population_size):
           if chose_parameter <= cumulative_probability[i]:
               new_chromosomes.append(chromosomes[i])
               break
    return new_chromosomes


## Crossover

In this step, we apply ```uniform single-point``` crossover. the single-point crossover means that the genes in two parents are swapped with one crossover line\
we also used two other methods for cross over:
* cross over with ``tournament selection`` and rate = 1 -> its rate was not flexible.
* cross over with ``all possible pairs`` from the selected set with rate 0.25 -> population increase very fast after generations.

In [27]:
def cross_over(chromosomes):
    if degree > 0:
        random.shuffle(chromosomes)
        for i in range(0, len(chromosomes), 2):
            if does_it_cross_over():
                line_ind = random.randint(1, degree)
                ch1 = chromosomes[i][0 : line_ind] + chromosomes[i + 1][line_ind : degree + 1]
                ch2 = chromosomes[i + 1][0 : line_ind] + chromosomes[i][line_ind : degree + 1]
                chromosomes[i] = ch1
                chromosomes[i + 1] = ch2
            
def does_it_cross_over():
    return random.uniform(0,1) <= cross_over_rate

Mutation is a process in which we assign new values to any genes. In this case, we use ``random mutation``. The number of genes that have mutations is determined by the mutation rate ``mutation_rate`` ($𝑃_{𝑚}$)

## Mutation

In [28]:
def mutation(chromosomes):
    for ch in chromosomes:
        if does_it_mutate():
            gen_ind = random.randint(0, degree)
            new_gen = random.randint(coef_range[0], coef_range[1])
            ch[gen_ind] = new_gen

def does_it_mutate():
    return random.uniform(0, 1) <= mutation_rate

## Evaluation

The process is repeated until we reached ``num_of_gens`` or find ``perfect result`` , then we print the chromosome with best fitness we ``ever seen`` as a result.

In [29]:
def evaluation(gen_num, chromosomes, best_fitness, best_chromosome):
    total, ch_fitnesses, new_best_fitness, new_best_chromosome = calculate_chs_fitness(chromosomes, best_fitness, best_chromosome)
    if(gen_num >= num_of_gens or best_fitness == perfect_result):
        print_result(best_chromosome)
        return False ,total, ch_fitnesses, new_best_fitness, new_best_chromosome
    else:
        return True, total, ch_fitnesses, new_best_fitness, new_best_chromosome

def calculate_chs_fitness(chromosomes, best_fitness, best_chromosome):
    total = 0
    ch_fitnesses = []
    new_best_fitness = best_fitness
    new_best_chromosome = best_chromosome
    for ch in chromosomes:
        fitness = check_fitness(ch)
        total += fitness
        ch_fitnesses.append(fitness)
        if fitness >= best_fitness:
            new_best_fitness = fitness
            new_best_chromosome = ch.copy()
    return total, ch_fitnesses, new_best_fitness, new_best_chromosome

def print_result(best_chromosome):
    result = ""
    for i in range(degree, -1, -1):
        if i == 0:
            result += str(best_chromosome[i])
        else:
            result += str(best_chromosome[i]) + " X^" + str(i) + " + "
    print(result)

## Main loop of GA

In [30]:
chromosomes = initialize_pop()
gen_num = 0
do_continue = True
best_chromosome = []
best_fitness = 0
while( do_continue ):
    do_continue , total_fit, ch_fitnesses, best_fitness, best_chromosome = evaluation(gen_num, chromosomes, best_fitness, best_chromosome)
    if( do_continue == False):
        break
    chromosomes = selection(chromosomes, total_fit, ch_fitnesses)
    cross_over(chromosomes)
    mutation(chromosomes)
    gen_num += 1


1 X^2 + 2 X^1 + 1


## Questions

### 1. How can very large or very small population make problem?
If the population is very large, the program will slow down, and in the selection step, the effect that the correctness of answer can have on its selection will be diminished. On a very small set, on average, we have to do many more searches to find the answer.

### 2. If the number of population increases in each period, what effect does it have on the accuracy and speed of the algorithm?
The amount of memory used increases, but the speed decreases. At the same time, this issue means testing more fitnesses and the possibility of finding a more optimal answer, but keeping the chromosomes that do not have enough accuracy means creating problems in convergence to the answer.

### 3. compare crossover and mutation? Is it possible to use only one of them?
In crossover we combine possible answers. But in mutation, we create completely new answers. Crossover operations for chromosomes that have passed the selection barrier are generally more reasonable than replacing them completely. Because the hope is to inherit the characteristics of both parents, but to create a chromosome that has a completely new desirable characteristic that has not been observed in previous chromosomes, mutation is definitely a faster method.


### 4. In your opinion, what solutions are there to get the answer to this particular problem faster?
The algorithm ``parameters`` are the most important factor they should be set carefully. \
then the ``fitness function`` is important And finally, optimizing the methods used for ``mutation`` and ``crossover``.

### 5. Despite the use of these methods, it is still possible that the chromosomes change after a few more steps don't do Explain the reason for this and the problems it causes. What do you suggest to solve it?
In some cases, we get involved in local maxima. In these times, mutation with new changes in chromosomes can cause us to get out of this state. Can we increase the mutation rate or start the algorithm all over again.

### 6. How to stop the algorithm if there exists no solution?
Before starting, check the conditions that cause this issue (for example, two cases with the same x and different y) or start the algorithm all over again (multi-start) in case of not finding a solution in specific number of generations. After that we can assume that probably there is no solution.

### 7. As the degree of polynomials increases, how does the time to find coefficients change?
Increasing the degree of polynomial means increasing the number of genes in each chromosome. Increasing it slows down the program. Our code is of order n in most areas.

### 8. In your opinion, how does increasing or decreasing the number of points affect the implementation of the algorithm?
By increasing the number of points, we draw the curve more clearly, which makes it less likely to be involved in localization maxima and also guides us towards a ``unique`` solution.